# RF: Genie3 vs. Arboreto equivalence test

## Conclusions:

* Genie3 uses `compute_feature_importances(normalize=False)`, but afterwards normalizes by dividing by the sum of the variable importances in the regression, which is equivalent to simply using `compute_feature_importances(normalize=True)`.
* Arboreto uses the `feature_importances_` attribute, which is equivalent to `compute_feature_importances(normalize=True)`, so the importances are equivalent to GENIE3, except perhaps some rounding errors.

In [1]:
import os
import sys
sys.path.append('../../')

from arboreto.core import *
from arboreto.utils import *

## Data paths

In [2]:
wd = os.getcwd().split('arboreto')[0] + 'arboreto/resources/dream5/'

net1_ex_path = wd + 'net1/net1_expression_data.tsv'
net1_tf_path = wd + 'net1/net1_transcription_factors.tsv'

net3_ex_path = wd + 'net3/net3_expression_data.tsv'
net3_tf_path = wd + 'net3/net3_transcription_factors.tsv'

net4_ex_path = wd + 'net4/net4_expression_data.tsv'
net4_tf_path = wd + 'net4/net4_transcription_factors.tsv'

## Sanity check wrt. GENIE3

In [3]:
m = load_expression_matrix(net1_ex_path)
g = load_gene_names(net1_ex_path)
t = load_tf_names(net1_tf_path, g)

In [4]:
tf_m = to_tf_matrix(expression_matrix=m, gene_names=g, tf_names=t)

In [99]:
from sklearn.tree.tree import BaseDecisionTree
def compute_feature_importances(estimator):
    """Computes variable importances from a trained tree-based model.
    """

    if isinstance(estimator, BaseDecisionTree):
        return estimator.tree_.compute_feature_importances(normalize=False)
    else:
        importances = [e.tree_.compute_feature_importances(normalize=False)
                       for e in estimator.estimators_]
        importances = np.asarray(importances)
        return np.sum(importances, axis=0) / len(estimator)

In [100]:
def to_links_df_2(regressor_type,
                  trained_regressor,
                  tf_names,
                  target_gene_name):
    """
    :param regressor_type: string. Case insensitive.
    :param trained_regressor: the trained model from which to extract the feature importances.
    :param tf_names: the list of names corresponding to the columns of the tf_matrix used to train the model.
    :param target_gene_name: the name of the target gene.
    :return: a Pandas DataFrame['TF', 'target', 'importance'] representing inferred regulatory links and their
             connection strength.
    """

    def pythonic():
        feature_importances = compute_feature_importances(trained_regressor)

        links_df = pd.DataFrame({'TF': tf_names, 'importance': feature_importances})
        links_df['target'] = target_gene_name

        clean_links_df = links_df[links_df.importance > 0].sort_values(by='importance', ascending=False)

        return clean_links_df[['TF', 'target', 'importance']]

    if is_pythonic_regressor(regressor_type):
        return pythonic()
    elif is_xgboost_regressor(regressor_type):
        raise ValueError('XGB regressor not yet supported')
    else:
        raise ValueError('Unsupported regressor type: ' + regressor_type)

In [101]:
def infer_data_2(regressor_type,
                 regressor_kwargs,
                 tf_matrix,
                 tf_names,
                 target_gene_name,
                 target_gene_expression,
                 include_meta=False,
                 early_stop_window_length=EARLY_STOP_WINDOW_LENGTH,
                 seed=DEMON_SEED):
    """
    Ties together regressor model training with regulatory links and meta data extraction.

    :param regressor_type: string. Case insensitive.
    :param regressor_kwargs: dict of key-value pairs that configures the regressor.
    :param tf_matrix: numpy matrix. The feature matrix X to use for the regression.
    :param tf_names: list of transcription factor names corresponding to the columns of the tf_matrix used to train the
                     regression model.
    :param target_gene_name: the name of the target gene to infer the regulatory links for.
    :param target_gene_expression: the expression profile of the target gene. Numpy array.
    :param include_meta: whether to also return the meta information DataFrame.
    :param early_stop_window_length: window length of the early stopping monitor.
    :param seed: (optional) random seed for the regressors.
    :return: if include_meta == True, return links_df, meta_df

             link_df: a Pandas DataFrame['TF', 'target', 'importance'] containing inferred regulatory links and their
             connection strength.

             meta_df: a Pandas DataFrame['target', 'meta', 'value'] containing meta information regarding the trained
             regression model.
    """

    (clean_tf_matrix, clean_tf_names) = clean(tf_matrix, tf_names, target_gene_name)

    trained_regressor = fit_model(regressor_type, regressor_kwargs, clean_tf_matrix, target_gene_expression,
                                  early_stop_window_length, seed)

    links_df = to_links_df_2(regressor_type, trained_regressor, clean_tf_names, target_gene_name)

    if include_meta:
        meta_df = to_meta_df(trained_regressor, target_gene_name)

        return links_df, meta_df
    else:
        return links_df

In [102]:
def sanity_check_2(target_idx, r_type='RF', r_kwargs=RF_KWARGS):
    target_gene = g[target_idx]
    
    (clean_tfm, clean_t) = clean(tf_m, t, target_gene)
    
    return infer_data_2(
        tf_matrix=clean_tfm, 
        tf_names=clean_t, 
        target_gene_name=target_gene,
        target_gene_expression=m[:, target_idx], 
        regressor_type=r_type, 
        regressor_kwargs=r_kwargs, 
        seed=DEMON_SEED)

In [103]:
%%time
result_2 = sanity_check_2(0)

CPU times: user 10 s, sys: 28 ms, total: 10.1 s
Wall time: 10 s


In [104]:
result_2.sort_values(['importance'], ascending=False).head(20)

,TF,target,importance
165,G167,G1,0.005861
73,G75,G1,0.002031
94,G96,G1,0.001621
181,G183,G1,0.001579
171,G173,G1,0.001524
81,G83,G1,0.001432
91,G93,G1,0.001341
115,G117,G1,0.001013
68,G70,G1,0.001000
101,G103,G1,0.000999


In [108]:
0.005861 / result_2['importance'].sum()

0.07803480457007667

In [107]:
result_2['importance'].sum()

0.07510751173518626

In [82]:
def sanity_check(target_idx, r_type='RF', r_kwargs=RF_KWARGS):
    target_gene = g[target_idx]
    
    (clean_tfm, clean_t) = clean(tf_m, t, target_gene)
    
    return infer_data(
        tf_matrix=clean_tfm, 
        tf_names=clean_t, 
        target_gene_name=target_gene,
        target_gene_expression=m[:, target_idx], 
        regressor_type=r_type, 
        regressor_kwargs=r_kwargs, 
        seed=DEMON_SEED)

In [84]:
%%time
result = sanity_check(0)

CPU times: user 10.1 s, sys: 20 ms, total: 10.2 s
Wall time: 10.2 s


In [85]:
result.sort_values(['importance'], ascending=False).head(20)

,TF,target,importance
165,G167,G1,0.078012
73,G75,G1,0.027010
94,G96,G1,0.021570
181,G183,G1,0.021273
171,G173,G1,0.020282
81,G83,G1,0.018931
91,G93,G1,0.017593
115,G117,G1,0.013442
68,G70,G1,0.013370
101,G103,G1,0.013316


# GENIE3 reference

* GENIE3 returns a different result every run because no seed value is specified.
* Let's hack Arboreto to implement GENIE3's importances function

In [11]:
VANANH_genie3_net1 = '/media/tmo/data/work/datasets/WOTC/GENIE3/DREAM5_NetworkInference_Other1_Network1.txt'

In [12]:
import pandas as pd

In [20]:
VA_net1 = pd.read_csv(VANANH_genie3_net1, sep='\t', header=None)
VA_net1.columns = ['TF', 'target', 'importance']

In [31]:
VA_net1[VA_net1['target'] == 'G1'].head(20)

,TF,target,importance
436,G167,G1,0.085304
3938,G75,G1,0.024893
5712,G183,G1,0.019611
5879,G173,G1,0.019263
6460,G96,G1,0.018048
7258,G83,G1,0.016742
8364,G93,G1,0.015246
10109,G70,G1,0.013425
12100,G117,G1,0.011985
12820,G161,G1,0.011533
